In [24]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [14]:
load_dotenv(override=True)

openrouter_api_key = os.getenv("OPENROUTER_API_KEY")
base_url = "https://openrouter.ai/api/v1"
openai = OpenAI(api_key=openrouter_api_key, base_url=base_url)


messages = [
    {"role": "system", "content": "You are a helpful assistant that can answer questions and help with tasks."},
    {"role": "user", "content": "Funny fact about kenya"}
]

response = openai.chat.completions.create(
    model="anthropic/claude-haiku-4-5",
    messages=messages
)

print(response.choices[0].message.content)

# A Funny Fact About Kenya 🦁

Kenya is home to the **Giraffe Centre in Nairobi**, where you can hand-feed giraffes that are taller than buildings—and they'll eat food right out of your hand with their 20-inch tongues. 

The funny part? Visitors often describe it as strangely intimate and awkward having a massive, gentle giant delicately lick your palm. Some people say it feels like being kissed by an alien with a 6-foot-long tongue! 😄

**Bonus funny fact:** Kenya has a desert that's inexplicably lush and green—it's called **Turkana Lake** and is located in one of the hottest, driest regions of the country, earning it the nickname "the Jade Sea." It's basically nature's practical joke.

Would you like to know more about Kenya's wildlife or culture?


### Simple chat bot that keeps conversation History

In [ ]:

def chat(message, history):

    print("message is:", message)
    print("history is:", history)
    system_message = """You are a helpful assistant in a clothes store. You should try to gently encourage \
                        the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
                        For example, if the customer says 'I'm looking to buy a hat', \
                        you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event e.g.,Kofia and Godfather.'\
                        Encourage the customer to buy hats if they are unsure what to get.
                        """
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
    return response.choices[0].message.content





In [ ]:
gr.ChatInterface(fn=chat).launch()

### Using yield to ensure proper streaming

In [ ]:
system_message = """You are a helpful assistant in a clothes store. You should try to gently encourage \
                        the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
                        For example, if the customer says 'I'm looking to buy a hat', \
                        you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event e.g.,Kofia and Godfather.'\
                        Encourage the customer to buy hats if they are unsure what to get.
                        """
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    relevant_system_message = system_message
    if 'belt' in message.lower():
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model="anthropic/claude-haiku-4-5", messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat).launch()

### An implementation to help visualiza entire conversation so far

In [ ]:
import gradio as gr


def history_to_markdown(history) -> str:
    """Render Gradio chat history into a readable transcript."""
    if not history:
        return "_No conversation yet._"

    lines = ["## Conversation history\n"]

    # Gradio may store history as either:
    # - messages format: [{"role": "user"|"assistant", "content": "..."}, ...]
    # - tuples format: [(user, assistant), ...]

    if history and isinstance(history[0], dict):
        for msg in history:
            role = str(msg.get("role", "")).strip() or "message"
            content = "" if msg.get("content") is None else str(msg.get("content"))
            lines.append(f"- **{role}**: {content}")
        return "\n".join(lines)

    for i, turn in enumerate(history, start=1):
        if isinstance(turn, (list, tuple)) and len(turn) == 2:
            user_msg, assistant_msg = turn
            user_msg = "" if user_msg is None else str(user_msg)
            assistant_msg = "" if assistant_msg is None else str(assistant_msg)
            lines.append(f"### Turn {i}\n**User:** {user_msg}\n\n**Assistant:** {assistant_msg}\n")
        else:
            lines.append(f"- {turn}")

    return "\n".join(lines)


def respond(message: str, history):
    if not (message or "").strip():
        return "", history, history_to_markdown(history)

    if "openai" not in globals() or openai is None:
        raise gr.Error("OpenAI/OpenRouter client not initialized. Run the earlier setup cell first.")

    relevant_system_message = system_message
    if "belt" in message.lower():
        relevant_system_message += (
            " The store does not sell belts; if asked for belts, suggest other items on sale instead."
        )

    messages = [{"role": "system", "content": relevant_system_message}]

    history = history or []

    # Normalize history into messages format for both Gradio + OpenAI
    normalized_history = []
    if history and isinstance(history[0], dict):
        normalized_history = [
            {"role": str(m.get("role", "")), "content": "" if m.get("content") is None else str(m.get("content"))}
            for m in history
        ]
    else:
        for user_msg, assistant_msg in history:
            if user_msg is not None and str(user_msg).strip() != "":
                normalized_history.append({"role": "user", "content": str(user_msg)})
            if assistant_msg is not None and str(assistant_msg).strip() != "":
                normalized_history.append({"role": "assistant", "content": str(assistant_msg)})

    messages.extend(normalized_history)
    messages.append({"role": "user", "content": message})

    response = openai.chat.completions.create(
        model="anthropic/claude-haiku-4-5",
        messages=messages,
    )
    assistant = response.choices[0].message.content

    new_history = normalized_history + [
        {"role": "user", "content": message},
        {"role": "assistant", "content": assistant},
    ]

    return "", new_history, history_to_markdown(new_history)


with gr.Blocks(title="Conversational AI (with transcript)") as demo:
    gr.Markdown("## Clothes store assistant\nChat below, then click **View conversation history** to see the full transcript.")

    try:
        chatbot = gr.Chatbot(label="Chat", type="messages")
    except TypeError:
        chatbot = gr.Chatbot(label="Chat")

    with gr.Row():
        msg = gr.Textbox(label="Message", placeholder="Ask about hats, shirts, etc.")

    with gr.Row():
        send = gr.Button("Send", variant="primary")
        view = gr.Button("View conversation history")
        clear = gr.Button("Clear")

    transcript = gr.Markdown(label="Transcript")

    send.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot, transcript])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot, transcript])
    view.click(history_to_markdown, inputs=[chatbot], outputs=[transcript])
    clear.click(lambda: ([], "", "_No conversation yet._"), outputs=[chatbot, msg, transcript])

demo.launch()